In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

In [0]:
sparkSession = SparkSession.builder.appName("Market Sales Spark Analysis").getOrCreate()

In [0]:
sparkSession

SparkSession - hive 
 
 
 SparkContext 

 Spark UI 

 
 Version 
 v3.1.2 
 Master 
 local[8] 
 AppName 
 Databricks Shell

In [0]:
rawDF = sparkSession.read.option("delimiter",";").csv('/FileStore/tables/MarketSales.csv',header=True,inferSchema=True).drop('SALESMAN','CLIENTNAME')

In [0]:
rawDF.printSchema()

root
 |-- ID: string (nullable = true)
 |-- ITEMNAME: string (nullable = true)
 |-- DATE_: string (nullable = true)
 |-- AMOUNT: double (nullable = true)
 |-- PRICE: double (nullable = true)
 |-- LINENETTOTAL: double (nullable = true)
 |-- BRANCHNR: integer (nullable = true)
 |-- BRANCH: string (nullable = true)
 |-- CITY: string (nullable = true)
 |-- REGION: string (nullable = true)
 |-- CLIENTCODE: string (nullable = true)
 |-- BRANDCODE: string (nullable = true)
 |-- BRAND: string (nullable = true)
 |-- CATEGORY_NAME1: string (nullable = true)
 |-- CATEGORY_NAME2: string (nullable = true)
 |-- CATEGORY_NAME3: string (nullable = true)
 |-- STARTDATE: string (nullable = true)
 |-- ENDDATE: string (nullable = true)
 |-- GENDER: string (nullable = true)



In [0]:
rawDF.describe().show()

+-------+--------------------+--------------------+--------------+------------------+-----------------+-----------------+------------------+-------------+------+----------+------------------+------------------+----------+--------------+---------------+-----------------+--------------+--------------+------+
|summary|                  ID|            ITEMNAME|         DATE_|            AMOUNT|            PRICE|     LINENETTOTAL|          BRANCHNR|       BRANCH|  CITY|    REGION|        CLIENTCODE|         BRANDCODE|     BRAND|CATEGORY_NAME1| CATEGORY_NAME2|   CATEGORY_NAME3|     STARTDATE|       ENDDATE|GENDER|
+-------+--------------------+--------------------+--------------+------------------+-----------------+-----------------+------------------+-------------+------+----------+------------------+------------------+----------+--------------+---------------+-----------------+--------------+--------------+------+
|  count|              611115|              603941|        611099|          

In [0]:
rawDF.printSchema()

root
 |-- ID: string (nullable = true)
 |-- ITEMNAME: string (nullable = true)
 |-- DATE_: string (nullable = true)
 |-- AMOUNT: double (nullable = true)
 |-- PRICE: double (nullable = true)
 |-- LINENETTOTAL: double (nullable = true)
 |-- BRANCHNR: integer (nullable = true)
 |-- BRANCH: string (nullable = true)
 |-- CITY: string (nullable = true)
 |-- REGION: string (nullable = true)
 |-- CLIENTCODE: string (nullable = true)
 |-- BRANDCODE: string (nullable = true)
 |-- BRAND: string (nullable = true)
 |-- CATEGORY_NAME1: string (nullable = true)
 |-- CATEGORY_NAME2: string (nullable = true)
 |-- CATEGORY_NAME3: string (nullable = true)
 |-- STARTDATE: string (nullable = true)
 |-- ENDDATE: string (nullable = true)
 |-- GENDER: string (nullable = true)



In [0]:
selectDF = rawDF.select('ITEMNAME','AMOUNT','PRICE','LINENETTOTAL','BRANCH')

In [0]:
selectDF.show()

+--------------------+------+-----+------------+----------------+
|            ITEMNAME|AMOUNT|PRICE|LINENETTOTAL|          BRANCH|
+--------------------+------+-----+------------+----------------+
|SPRITE 1 LT LIMON...|   1.0|  2.0|         2.0|  Kocaeli Subesi|
|           TOZ SEKER|   5.0| 2.65|       13.25|  Antalya Subesi|
|FALIM SAKIZ 5LI NANE|   1.0|  0.4|         0.4| �stanbul Subesi|
|FALIM SAKIZ 5LI NANE|   1.0|  0.4|         0.4| �stanbul Subesi|
|FALIM SAKIZ 5LI C...|   1.0|  0.4|         0.4|    �zmir Subesi|
|           TOZ SEKER|   2.0| 2.65|         5.3|   Elaz�� Subesi|
|F NEFFIS TOZ SEKE...|   1.0|  5.6|         5.6|  Malatya Subesi|
|           TOZ SEKER|   2.0| 2.65|         5.3| �stanbul Subesi|
|           TOZ SEKER|   2.0| 2.65|         5.3|   Manisa Subesi|
|F NEFFIS KESME SE...|   1.0| 2.85|        2.85|    Mu�la Subesi|
|KENT TOPITOP DISN...|   1.0|  1.6|         1.6|      Mu� Subesi|
|VIVIDENT CZD FRUI...|   1.0| 1.95|        1.95|    Adana Subesi|
|LAVACHE Q

In [0]:
selectDF.agg({'LINENETTOTAL':'sum'}).show()

+------------------+
| sum(LINENETTOTAL)|
+------------------+
|2478409.4400002426|
+------------------+



In [0]:
selectDF.agg({'LINENETTOTAL':'max'}).show()

+-----------------+
|max(LINENETTOTAL)|
+-----------------+
|           4278.0|
+-----------------+



In [0]:
sumTotalByBranchDF = selectDF.groupBy('BRANCH').sum('LINENETTOTAL').orderBy('sum(LINENETTOTAL)',ascending=False)

In [0]:
resultDF = sumTotalByBranchDF.withColumn('TOTALPRICE',round('sum(LINENETTOTAL)',2)).drop('sum(LINENETTOTAL)')

In [0]:
display(resultDF)

BRANCH,TOTALPRICE
�stanbul Subesi,458484.93
Ankara Subesi,167002.25
�zmir Subesi,131799.05
Bursa Subesi,81148.86
Antalya Subesi,75283.4
Adana Subesi,64980.84
Konya Subesi,63917.53
�anl�urfa Subesi,58235.57
Kocaeli Subesi,57708.94
Zonguldak Subesi,57570.73


In [0]:
selectDF.groupBy('ITEMNAME').count().orderBy('count',ascending=False).show(truncate=False)

+-----------------------------+-----+
|ITEMNAME                     |count|
+-----------------------------+-----+
|EKMEK 250 GR                 |9122 |
|null                         |7175 |
|DOMATES                      |6018 |
|PATATES                      |5718 |
|PORTAKAL                     |5611 |
|TOZ SEKER                    |5242 |
|SOGAN                        |4631 |
|OSMANCIK PIRINC KG.          |4101 |
|LIMON                        |4093 |
|MUZ                          |3934 |
|KIVIRCIK                     |3925 |
|MAYDANOZ                     |3808 |
|HAVUC                        |3632 |
|S�H�RL� ELLER C�G K�FTE 200GR|3566 |
|F NEFFIS S�T YARIM YA�LI 1 LT|3253 |
|YERLI BADEM                  |2900 |
|ULKER CIKOLATALI GOFRET 35GR |2692 |
|YAHYAOGLU 15 LI YUMURTA      |2588 |
|PORTAKAL SIKMALIK            |2561 |
|SALKIM DOMATES               |2471 |
+-----------------------------+-----+
only showing top 20 rows



In [0]:
#rawDF.select('ITEMNAME','LINENETTOTAL','CLIENTCODE').filter(("LINENETTOTAL>1500 and LINENETTOTAL<5000")).show()
nullwoclientCodeDF.select('ITEMNAME','LINENETTOTAL','CLIENTCODE').filter((rawDF['LINENETTOTAL']>1500) & (rawDF['LINENETTOTAL']<5000)).show()

+--------------------+------------+----------+
|            ITEMNAME|LINENETTOTAL|CLIENTCODE|
+--------------------+------------+----------+
|     SAMSUN 216 SOFT|      2761.0|      9010|
|     SAMSUN 216 SOFT|      2761.0|      9010|
|CANPED POLITENLI ...|      4278.0|    854889|
|CANPED POLITENLI ...|      4278.0|    854889|
+--------------------+------------+----------+



In [0]:
nullclientCodeDF = rawDF.filter(col('CLIENTCODE').isNull())

In [0]:
nullwoclientCodeDF = rawDF.filter(col('CLIENTCODE').isNotNull())

In [0]:
nullwoclientCodeDF.show()

+-----+--------------------+--------------+------+-----+------------+--------+----------------+---------+-----------------+----------+---------+--------------+---------------+---------------+------------------+--------------+--------------+------+
|   ID|            ITEMNAME|         DATE_|AMOUNT|PRICE|LINENETTOTAL|BRANCHNR|          BRANCH|     CITY|           REGION|CLIENTCODE|BRANDCODE|         BRAND| CATEGORY_NAME1| CATEGORY_NAME2|    CATEGORY_NAME3|     STARTDATE|       ENDDATE|GENDER|
+-----+--------------------+--------------+------+-----+------------+--------+----------------+---------+-----------------+----------+---------+--------------+---------------+---------------+------------------+--------------+--------------+------+
|11738|SPRITE 1 LT LIMON...|07/01/17 00:00|   1.0|  2.0|         2.0|      52|  Kocaeli Subesi|  Kocaeli|          Marmara|    467369|      156|        SPR�TE|         ��ECEK|   GAZLI ��ECEK|             GAZOZ|08/01/17 16:16|08/01/17 16:17|     E|
|10537| 

In [0]:
rawDF.select('BRANCH').withColumn('NEWBRANCH',trim(regexp_replace('BRANCH','Subesi','Mağaza'))).drop('BRANCH').show()

+----------------+
|       NEWBRANCH|
+----------------+
|  Kocaeli Mağaza|
|  Antalya Mağaza|
| �stanbul Mağaza|
| �stanbul Mağaza|
|    �zmir Mağaza|
|   Elaz�� Mağaza|
|  Malatya Mağaza|
| �stanbul Mağaza|
|   Manisa Mağaza|
|    Mu�la Mağaza|
|      Mu� Mağaza|
|    Adana Mağaza|
|  Sakarya Mağaza|
|Zonguldak Mağaza|
|Zonguldak Mağaza|
|Bal�kesir Mağaza|
|Bal�kesir Mağaza|
|K�r�kkale Mağaza|
|Gaziantep Mağaza|
|Gaziantep Mağaza|
+----------------+
only showing top 20 rows



In [0]:
rawDF.toPandas()